In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc_version = 'gcc6'
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

Using TensorFlow backend.


(2337112, 2) (1787504, 2) (1892514, 2) (2167171, 2)
reset_index 완료


,hex,bin
0,55,1
1,89,0
2,e5,0
3,56,0
4,53,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['hex'].unique()),len(gcc6_1_32['hex'].unique()))
print(len(gcc6_2_32['hex'].unique()),len(gcc6_3_32['hex'].unique()))

256 256
256 256


In [3]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2337112, 257) (1787504, 257) (1892514, 257) (2167171, 257)


In [4]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['bin'].value_counts(), '\n', gcc6_1_32_onehot['bin'].value_counts())
print(gcc6_2_32_onehot['bin'].value_counts(), '\n', gcc6_3_32_onehot['bin'].value_counts())

0    2324953
1      12159
Name: bin, dtype: int64 
 0    1776389
1      11115
Name: bin, dtype: int64
0    1881573
1      10941
Name: bin, dtype: int64 
 0    2156304
1      10867
Name: bin, dtype: int64


In [5]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['hex'].value_counts()

00    323587
ff    255657
8b    153094
83    132575
45     92567
       ...  
a2       294
91       290
ab       290
9e       277
a9       234
Name: hex, Length: 256, dtype: int64

In [7]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['bin']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
left_idx, right_idx = -10, 11 # 10개씩
# left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 199597


In [8]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['bin']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 169245


In [9]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['bin']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 179621


In [10]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['bin']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 177813


In [11]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['bin'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['bin'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['bin'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['bin'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(199597, 256) (199597,)
(169245, 256) (169245,)
(179621, 256) (179621,)
(177813, 256) (177813,)


In [12]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(199597, 1, 256) (199597, 1, 1)
(169245, 1, 256) (169245, 1, 1)
(179621, 1, 256) (179621, 1, 1)
(177813, 1, 256) (177813, 1, 1)


In [13]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc6_0123_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_0123_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_0123_32.shape, y_gcc6_0123_32.shape)

(726276, 1, 256) (726276, 1, 1)


In [14]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(x_gcc6_0123_32, y_gcc6_0123_32):
    
    xInput = Input(batch_shape=(None, 1, 256)) # 
    xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
    xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #print(model1.summary())
    print('======Training stage======')
    model1.fit(x_gcc6_0123_32[train],
               y_gcc6_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(x_gcc6_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
653648/653648 [==============================] - 49s 75us/step - loss: 0.1225 - accuracy: 0.9467
Epoch 2/3
653648/653648 [==============================] - 54s 83us/step - loss: 0.1185 - accuracy: 0.9473
Epoch 3/3
653648/653648 [==============================] - 54s 83us/step - loss: 0.1191 - accuracy: 0.9472


accuracy_score 0.96439389767032
recall_score 0.572013302634945
precision_score 0.7100666878374087
f1_score 0.6336072541796542
======Training stage======
Epoch 1/3
653648/653648 [==============================] - 51s 78us/step - loss: 0.1231 - accuracy: 0.9467
Epoch 2/3
653648/653648 [==============================] - 57s 87us/step - loss: 0.1182 - accuracy: 0.9473
Epoch 3/3
653648/653648 [==============================] - 54s 83us/step - loss: 0.1186 - accuracy: 0.9471


accuracy_score 0.9483119458060252
recall_score 0.25349176568688764
precision_score 0.8754499640028798
f1_score 0.3931458131264145
======Training stage======
Epoch 1/3
653648/653648 [==============================] - 62s 95us/step - loss: 0.1217 - accuracy: 0.9465
Epoch 2/3
653648/653648 [==============================] - 55s 85us/step - loss: 0.1169 - accuracy: 0.9470
Epoch 3/3
653648/653648 [==============================] - 51s 78us/step - loss: 0.1173 - accuracy: 0.9471


accuracy_score 0.9542876025775183
recall_score 0.28909740840035747
precision_score 0.9036312849162011
f1_score 0.4380501015572106
======Training stage======
Epoch 1/3
653648/653648 [==============================] - 63s 96us/step - loss: 0.1188 - accuracy: 0.9483
Epoch 2/3
653648/653648 [==============================] - 53s 81us/step - loss: 0.1142 - accuracy: 0.9486
Epoch 3/3
653648/653648 [==============================] - 60s 91us/step - loss: 0.1135 - accuracy: 0.9486


accuracy_score 0.949785206807292
recall_score 0.22723926380368098
precision_score 0.6502808988764045
f1_score 0.33678850700127294
======Training stage======
Epoch 1/3
653648/653648 [==============================] - 60s 92us/step - loss: 0.1212 - accuracy: 0.9494
Epoch 2/3
653648/653648 [==============================] - 52s 79us/step - loss: 0.1168 - accuracy: 0.9498
Epoch 3/3
653648/653648 [==============================] - 62s 94us/step - loss: 0.1170 - accuracy: 0.9498


...

accuracy_score 0.9214077215399019
recall_score 0.2434632461766157
precision_score 0.27205071664829106
f1_score 0.2569643322051549
======Training stage======
Epoch 1/3
653649/653649 [==============================] - 56s 86us/step - loss: 0.1207 - accuracy: 0.9490
Epoch 2/3
653649/653649 [==============================] - 63s 97us/step - loss: 0.1156 - accuracy: 0.9495
Epoch 3/3
653649/653649 [==============================] - 55s 84us/step - loss: 0.1156 - accuracy: 0.9492


accuracy_score 0.9357676897021768
recall_score 0.06558704453441296
precision_score 0.868632707774799
f1_score 0.12196499153020891
======Training stage======
Epoch 1/3
653649/653649 [==============================] - 54s 83us/step - loss: 0.1185 - accuracy: 0.9507
Epoch 2/3
653649/653649 [==============================] - 67s 103us/step - loss: 0.1133 - accuracy: 0.9513
Epoch 3/3
653649/653649 [==============================] - 55s 84us/step - loss: 0.1137 - accuracy: 0.9512


accuracy_score 0.9292274223085079
recall_score 0.39597478176527645
precision_score 0.3813638486688463
f1_score 0.38853200095170115
======Training stage======
Epoch 1/3
653649/653649 [==============================] - 71s 109us/step - loss: 0.1181 - accuracy: 0.9512
Epoch 2/3
653649/653649 [==============================] - 64s 98us/step - loss: 0.1134 - accuracy: 0.9516
Epoch 3/3
653649/653649 [==============================] - 62s 95us/step - loss: 0.1133 - accuracy: 0.9516


accuracy_score 0.9263221666873201
recall_score 0.3231546044722289
precision_score 0.3463917525773196
f1_score 0.33436994651076
======Training stage======
Epoch 1/3
653649/653649 [==============================] - 60s 91us/step - loss: 0.1203 - accuracy: 0.9490
Epoch 2/3
653649/653649 [==============================] - 61s 93us/step - loss: 0.1154 - accuracy: 0.9493
Epoch 3/3
653649/653649 [==============================] - 66s 101us/step - loss: 0.1162 - accuracy: 0.9494


accuracy_score 0.9444696875817533
recall_score 0.71643637863477
precision_score 0.5665362035225049
f1_score 0.6327292596302705

K-fold cross validation Accuracy: [0.96439389767032, 0.9483119458060252, 0.9542876025775183, 0.949785206807292, 0.9264333314974941, 0.9214077215399019, 0.9357676897021768, 0.9292274223085079, 0.9263221666873201, 0.9444696875817533]

K-fold cross validation Recall: [0.572013302634945, 0.25349176568688764, 0.28909740840035747, 0.22723926380368098, 0.06584723441615452, 0.2434632461766157, 0.06558704453441296, 0.39597478176527645, 0.3231546044722289, 0.71643637863477]

K-fold cross validation Precision: [0.7100666878374087, 0.8754499640028798, 0.9036312849162011, 0.6502808988764045, 0.9422110552763819, 0.27205071664829106, 0.868632707774799, 0.3813638486688463, 0.3463917525773196, 0.5665362035225049]

K-fold cross validation F1-Score: [0.6336072541796542, 0.3931458131264145, 0.4380501015572106, 0.33678850700127294, 0.12309207287050712, 0.2569643322051549, 0.121964

In [15]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.940040667217831
10-Fold Cross_validation. Recall : 0.315230503052533
10-Fold Cross_validation. Precision : 0.6516615120101037
10-Fold Cross_validation. F1-Score : 0.3659244279563155
